In [1]:
"""
Neural Network Policy using TensorFlow
- Takes observation as input and will output action to be  taken in the current state
- From Chapter 16 of Handson with Machine Learning using Scikitlearn and TensorFlow
- Uses OpenAI Gym

High Level Program Flow:
- On each training iteration:
  - run the policy for n_games_per_update (=10) episodes. Each episode has n_max_steps number of maximum steps
      - at each step of policy execution (pretending that action chose is the best) compute policy gradients
  - compute the action_scores using discount_and_normalize_rewards() functions
  - multiply each gradient vector by it corresponding action score across all acorss all episodes and steps
  - compute the mean of the resulting gradients
  - run the training operation, feeding the mean gradients
"""

'\nNeural Network Policy using TensorFlow\nFrom Chapter 16 \nUses OpenAI Gym\n'

In [2]:
import gym
env = gym.make('CartPole-v0')
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected

[2018-01-22 22:28:03,016] Making new env: CartPole-v0
Couldn't import dot_parser, loading of dot files will not be possible.


In [3]:
# 1. Specify the NN Architecture
n_inputs = 4 # == env.observation_space.shape[0]
n_hidden = 4
n_outputs = 1 # prob of accelerating left
initializer = tf.contrib.layers.variance_scaling_initializer()

# 2. Build the NN
X = tf.placeholder(tf.float32, shape=[None, n_inputs])
hidden = fully_connected(X, n_hidden, activation_fn=tf.nn.elu, weights_initializer=initializer)
logits = fully_connected(hidden, n_outputs, activation_fn=None, weights_initializer=initializer)
outputs = tf.nn.sigmoid(logits)

#3. Select a random action based on the estimated probabilities
p_left_and_right = tf.concat(values=[outputs, 1 - outputs], axis=1)
action = tf.multinomial(tf.log(p_left_and_right), num_samples=1)

init=tf.global_variables_initializer()

In [8]:
# Setting up the training of the NN Using Policy Gradient

y = 1.0 - tf.to_float(action) # target probability is 1 when action is 0 and 0 when action is 1
learning_rate = 0.01
cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=y)
optimizer = tf.train.AdamOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(cross_entropy)
gradients = [grad for grad, variable in grads_and_vars]
gradient_placeholders = []
grads_and_vars_feed = []
for grad, variable in grads_and_vars:
    gradient_placeholder = tf.placeholder(tf.float32)
    gradient_placeholders.append(gradient_placeholder)
    grads_and_vars_feed.append((gradient_placeholder, variable))
    
training_op = optimizer.apply_gradients(grads_and_vars_feed)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

#Function to compute the total discounted rewards given the raw rewards
def discount_rewards(rewards, discount_rate):
    discounted_rewards = np.empty(len(rewards))
    cumulative_rewards = 0
    for step in reversed(range(len(rewards))):
        cumulative_rewards = rewards[step] + discount_rate * cumulative_rewards
        discounted_rewards[step] = cumulative_rewards
    return discounted_rewards

def discount_and_normalize_rewards(all_rewards, discount_rate):
    all_discounted_rewards = [discount_rewards(rewards, discount_rate) for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean)/reward_std for discounted_rewards in all_discounted_rewards]

In [9]:
n_iterations = 250 # number of training iterations
n_max_steps = 1000 # max steps per episode
n_games_per_update = 10 # train the policy every 10 episodes
save_iterations = 10 # save the model every 10 training iterations
discount_rate = 0.95

In [10]:
# Executing the graph
with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        all_rewards = [] # all sequences of raw rewards for each episode
        all_gradients = [] # gradients saved at each step of each episode
        for game in range(n_games_per_update):
            current_rewards = [] # all raw rewards for the current episode
            current_gradients = [] # all gradients from the current episode
            obs = env.reset()
            for step in range(n_max_steps):
                action_val, gradients_val = sess.run([action, gradients], feed_dict={X: obs.reshape(1, n_inputs)})
                obs, reward, done, info = env.step(action_val[0][0])
                current_rewards.append(reward)
                current_gradients.append(gradients_val)
                if done:
                    break
            all_rewards.append(current_rewards)
            all_gradients.append(current_gradients)
            
            # Perform a Policy Update after running the policy for 10 episodes
            all_rewards = discount_and_normalize_rewards(all_rewards, discount_rate)
            feed_dict = {}
            for var_index, grad_placeholder in enumerate(gradient_placeholders):
                # Multiply the gradients by the action scores and compute the mean
                mean_gradients = np.mean([reward * all_gradients[game_index][step][var_index]
                                         for game_index, rewards in enumerate(all_rewards)
                                         for step, reward in enumerate(rewards)], axis=0)
                feed_dict[grad_placeholder] = mean_gradients
                
            sess.run(training_op, feed_dict=feed_dict)
            if iteration % save_iterations == 0:
                saver.save(sess, 'my_policy_net_pg.ckpt')


NameError: global name 'np' is not defined